# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4910, uptime 0:00:33
mini_beamline                    RUNNING   pid 4911, uptime 0:00:33
random_walk                      RUNNING   pid 4912, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4913, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4914, uptime 0:00:33
simple                           RUNNING   pid 4915, uptime 0:00:33
thermo_sim                       RUNNING   pid 4916, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4917, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-06 20:02:33
Persistent Unique Scan ID: 'c8c9191d-6bf1-4f20-a055-f304d840274f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:02:33.6 |     -1.000 |      99518 |
|         2 | 20:02:33.6 |     -0.500 |     100864 |
|         3 | 20:02:33.6 |      0.000 |     101633 |
|         4 | 20:02:33.7 |      0.500 |     100685 |
|         5 | 20:02:33.7 |      1.000 |      97589 |


+-----------+------------+------------+------------+
generator scan ['c8c9191d'] (scan num: 3)





('c8c9191d-6bf1-4f20-a055-f304d840274f',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/c8c9191d-6bf1-4f20-a055-f304d840274f-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-06 20:02:33
Persistent Unique Scan ID: '88001110-b15c-4655-92b1-335d527292ca'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:02:34.1 |          -1 | 1082632161 |


|         2 | 20:02:34.2 |          -0 | 1074564968 |


|         3 | 20:02:34.3 |           0 | 1064216535 |
|         4 | 20:02:34.4 |           0 | 1059853633 |


|         5 | 20:02:34.5 |           1 | 1056812339 |
+-----------+------------+-------------+------------+
generator scan ['88001110'] (scan num: 4)





('88001110-b15c-4655-92b1-335d527292ca',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-06 20:02:34
Persistent Unique Scan ID: '8b544f44-0e7d-4a8a-aba7-faaaf2d710ac'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:02:35.0 | 1054054344 |


|         2 | 20:02:35.9 | 1100861110 |


|         3 | 20:02:36.9 | 1130800442 |


...

|        28 | 20:03:02.0 | 1080423482 |


|        29 | 20:03:03.0 | 1053833568 |


|        30 | 20:03:04.0 | 1103415416 |


+-----------+------------+------------+
generator count ['8b544f44'] (scan num: 5)





('8b544f44-0e7d-4a8a-aba7-faaaf2d710ac',)